# ABOUT


Datascientest's Datascientist continuous bootcamp - cohorte Mars2022 -  AeroBOT project

**Tutor**

* Alban THUET

**Authors:**

* Hélène ASSIR 
* [Ioannis STASINOPOULOS](https://www.linkedin.com/in/ioannis-stasinopoulos/)

</br>

---
</br>

**Version History**

Version | Date       | Author(s)  | Modification
--------|----------- | ---------  | --------------------------
1.0     | 23/10/2022 | I.S        | Adapt 04.1.2 to load, preprocess and save the FINAL TEST set into a .pkl

This notebook

* mounts the GDrive of our AeroBot project @gmail account.

* loads the data from the `test_data_final.pkl` file (cf. `0_test_set_creator_DO_NOT_MODIFY_20220630.ipynb`) under a pandas DataFrame named `df`, which contains `10824` entries and 96 columns.
These data do not contain any more UAS-related entries.

* drops all columns, except, ACN (index), Anomaly, Narrative

* Performs Feature Engineering: one-hot-encoding of Anomaly using a naive approache, looking directly for the root label into the multiple modalities string of Anomaly

Further processing of the data should be perfomed on this dataframe, after having **copied a version of the present notebook**.

# IMPORT PACKAGES


settings for  full / patial Narrative display. Helene?


In [1]:
#######################
# Import packages
#######################
import numpy as np
import seaborn as sns
import math # for math.pi etc.
import time # time code execution

#######################
# Pandas
#######################
import pandas as pd
# Set pandas settings to show all data when using .head(), .columns etc.
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option("display.colheader_justify","left") # left-justify the print output of pandas

### Display full columnwidth
# Set pandas settings to display full text columns
#pd.options.display.max_colwidth = None
# Restore pandas settings to display standard colwidth
pd.reset_option('display.max_colwidth')

import itertools # Pour créer des iterateurs

# Package to show the progression of pandas operations
from tqdm import tqdm
# from tqdm.auto import tqdm  # for notebooks

# Create new `pandas` methods which use `tqdm` progress
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()
# simply use .progress_apply() instead of .apply() on your pd.DataFram

######################
# PLOTTING
######################
import matplotlib.pyplot as plt
%matplotlib inline
# # Define global plot parameters for better readability and consistency among plots
# # A complete list of the rcParams keys can be retrieved via plt.rcParams.keys() function
# plt.rcParams['axes.titlesize'] = 30
# plt.rcParams['axes.labelsize'] = 23
# plt.rcParams['xtick.labelsize'] = 23
# plt.rcParams['ytick.labelsize'] = 23
# plt.rc('legend', fontsize=23)    # legend fontsize

# BOKEH 
from bokeh.plotting import figure # Importation de la classe figure qui permet de créer un graphique bokeh.
from bokeh.io import  push_notebook, output_notebook, show
output_notebook() # permet d'afficher tous les futurs graphiques dans l'output d'une cellule jupyter. Si cette instruction n'est pas lancée, la figure s'affichera dans un nouvel onglet.
from bokeh.models import ColumnDataSource, Label
from bokeh.transform import dodge
from bokeh.models.tools import HoverTool

#####################
# NLP 
#####################
import re # for Regular Expression handling
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') # WordNet lemmatizer
nltk.download('omw-1.4') # necessary for WordNet lemmatizer
from nltk.tokenize import word_tokenize # Usual tokenizer
from nltk.tokenize import TweetTokenizer # Special tokenizer;  "we'll", "didn't", etc. are considered as one word
from sklearn.feature_extraction.text import CountVectorizer # Vectorization
from nltk.corpus import stopwords # Import stopwords from nltk.corpus
from nltk.stem.snowball import EnglishStemmer
##############################
# Undersampeling
##############################
from imblearn.under_sampling import RandomUnderSampler 
###############################
# ML preprocessing and models
###############################
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import ensemble # random forest
from sklearn.svm import SVC

from sklearn.metrics import classification_report, confusion_matrix


###############################
# Other
###############################
import pickle as pkl # Saving data externally

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


# LOAD DATA

## Mount GDrive

In [2]:
#@title
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive/')

#check your present working directory 
%pwd

Mounted at /content/drive/


'/content'

In [3]:
#@title
# move to the transformed data location (you can create a deeper structure, if needed, e.g. to save a trained model):
%cd /content/drive/MyDrive/data/transformed/

/content/drive/MyDrive/data/transformed


In [4]:
#@title
!ls # list the content of the pwd

#!ls "/content/drive/MyDrive/Data_Science/Formations/DataScienceTest/projet/AeroBot/" # list contect of a speficic folder

 2022_09_11_7_4_3_raw_narr_BERT_BASE_frozen_max_length_345.pkl
 complaints-2022-08-05_13_55.csv
'Copy of Qualified abbreviations_20220718.xlsx.gsheet'
'Data Dictionnary.xlsx'
 data_for_BERT_multilabel_20220805.pkl
 df_for_Anomaly_prediction.pkl
 model.png
 model_results
 Narrative_PP_stemmed_24072022_TRAIN.pkl
 Narrative_Raw_Stemmed_24072022_TRAIN.pkl
 Narrative_RegEx_subst_21072022_TRAIN.pkl
'Qualified abbreviations_20220707_test.csv'
'Qualified abbreviations_20220708.csv'
'Qualified abbreviations_20220718.csv'
'Qualified abbreviations_20220718_Google_sheet.gsheet'
 test_data_final.pkl
 train_data_final.pkl


## Load data from .pkl file


In [5]:
#@title
# Load the TRAIN data (97417 entries)
# Do not touch the TEST data until the end of the project!
# or the curse of the greek gods will fall upon you!

%cd /content/drive/MyDrive/data/transformed/
with open("test_data_final.pkl", "rb") as f:
    loaded_data = pkl.load(f)

df = loaded_data[0]
print("\nA Dataframe with", len(df), "entries has been loaded")

/content/drive/MyDrive/data/transformed

A Dataframe with 10824 entries has been loaded


# Delete unnecessary columns

In [6]:
columns_to_keep = ['Narrative', 'Anomaly']
df = df[columns_to_keep]

In [7]:
df.head()

,Narrative,Anomaly
ACN,,
1014798,Flying into SLC on the DELTA THREE RNAV arriva...,Aircraft Equipment Problem Less Severe; Deviat...
1806744,ORD was on a very busy east flow arrival push....,ATC Issue All Types; Conflict NMAC; Deviation ...
1044902,B737-800 was vectored to an ILS Runway 16L app...,ATC Issue All Types; Deviation / Discrepancy -...
1764093,We were on a 6 mile final when tower cleared a...,ATC Issue All Types; Conflict Ground Conflict;...
1786435,During Climb we Leveled at 17;000 departure sw...,ATC Issue All Types; Deviation - Altitude Excu...


# TARGET FEATURE ENGINEERING

## Anomaly

This section 

*   drops the entries with NaN in 'Anomaly'
*   creates and adds to the DataFrame 'df' a one-hot column for each Anomaly root label

Root labels are as defined in the ASRS coding form.pdf

We use a naive approach using string parsing 

### Drop the NaNs in Anomaly

In [8]:
# Drop Anomaly NaN's
print(f"We have {len(df[df['Anomaly'].isna()])} entries where 'Anomaly' == NaN. We drop these entries")

df = df.dropna(axis = 0, how = 'any', subset = ['Anomaly'])
print(f"Current length of our DataFrame: {len(df)}")

We have 19 entries where 'Anomaly' == NaN. We drop these entries
Current length of our DataFrame: 10805


### One-hot encode Anomaly root labels

In [9]:
# Root label (source = ASRS coding forms) : order = by descending frequency
Anomaly_RootLabels=['Deviation / Discrepancy - Procedural',
                    'Aircraft Equipment',
                    'Conflict',
                    'Inflight Event / Encounter',
                    'ATC Issue',
                    'Deviation - Altitude',
                    'Deviation - Track / Heading',
                    'Ground Event / Encounter',
                    'Flight Deck / Cabin / Aircraft Event',
                    'Ground Incursion',
                    'Airspace Violation',
                    'Deviation - Speed',
                    'Ground Excursion',
                    'No Specific Anomaly Occurred']

#### Naive one-hot encoding

In [10]:
# Create a column in the df corresponding to each Anomaly root label 'anomaly_rl'
Anomaly_RootLabels_columns = []
for anomaly_rl in Anomaly_RootLabels:
    col='Anomaly_' + anomaly_rl
    Anomaly_RootLabels_columns.append(col)
    # Fill the columns in a one-hot-encoding logic
    df[col] = df['Anomaly'].astype('str').apply(lambda x: 1 if (anomaly_rl in x)  else 0)

In [11]:
for anomaly_rl_col in Anomaly_RootLabels_columns:
    print('\n',anomaly_rl_col,'\n',df[anomaly_rl_col].value_counts())


 Anomaly_Deviation / Discrepancy - Procedural 
 1    6343
0    4462
Name: Anomaly_Deviation / Discrepancy - Procedural, dtype: int64

 Anomaly_Aircraft Equipment 
 0    6454
1    4351
Name: Anomaly_Aircraft Equipment, dtype: int64

 Anomaly_Conflict 
 0    8926
1    1879
Name: Anomaly_Conflict, dtype: int64

 Anomaly_Inflight Event / Encounter 
 0    8382
1    2423
Name: Anomaly_Inflight Event / Encounter, dtype: int64

 Anomaly_ATC Issue 
 0    8283
1    2522
Name: Anomaly_ATC Issue, dtype: int64

 Anomaly_Deviation - Altitude 
 0    9843
1     962
Name: Anomaly_Deviation - Altitude, dtype: int64

 Anomaly_Deviation - Track / Heading 
 0    9931
1     874
Name: Anomaly_Deviation - Track / Heading, dtype: int64

 Anomaly_Ground Event / Encounter 
 0    9827
1     978
Name: Anomaly_Ground Event / Encounter, dtype: int64

 Anomaly_Flight Deck / Cabin / Aircraft Event 
 0    9897
1     908
Name: Anomaly_Flight Deck / Cabin / Aircraft Event, dtype: int64

 Anomaly_Ground Incursion 
 0    

In [12]:
df.head()

,Narrative,Anomaly,Anomaly_Deviation / Discrepancy - Procedural,Anomaly_Aircraft Equipment,Anomaly_Conflict,Anomaly_Inflight Event / Encounter,Anomaly_ATC Issue,Anomaly_Deviation - Altitude,Anomaly_Deviation - Track / Heading,Anomaly_Ground Event / Encounter,Anomaly_Flight Deck / Cabin / Aircraft Event,Anomaly_Ground Incursion,Anomaly_Airspace Violation,Anomaly_Deviation - Speed,Anomaly_Ground Excursion,Anomaly_No Specific Anomaly Occurred
ACN,,,,,,,,,,,,,,,,
1014798,Flying into SLC on the DELTA THREE RNAV arriva...,Aircraft Equipment Problem Less Severe; Deviat...,1,1,0,0,0,1,0,0,0,0,0,0,0,0
1806744,ORD was on a very busy east flow arrival push....,ATC Issue All Types; Conflict NMAC; Deviation ...,1,0,1,0,1,0,0,0,0,0,0,0,0,0
1044902,B737-800 was vectored to an ILS Runway 16L app...,ATC Issue All Types; Deviation / Discrepancy -...,1,0,0,0,1,0,0,0,0,0,0,0,0,0
1764093,We were on a 6 mile final when tower cleared a...,ATC Issue All Types; Conflict Ground Conflict;...,1,0,1,0,1,0,0,0,0,0,0,1,0,0
1786435,During Climb we Leveled at 17;000 departure sw...,ATC Issue All Types; Deviation - Altitude Excu...,1,0,0,0,1,1,0,0,0,0,0,0,0,0


# Save the data

In [13]:
# ##########################################################
# # WARNING!! 
# # If you execute this cell, you will OVERWRITTE the data!
# ##########################################################

# %cd /content/drive/MyDrive/data/transformed/

# # save the df['Narrative_RegEx_subst'] externally to avoid having to perform the 
# # substitutions again
# with open("df_test_for_Anomaly_prediction.pkl", "wb") as f:
#     pkl.dump(df, f) # saves the variables into a list

/content/drive/MyDrive/data/transformed
